In [ ]:
import os
import sys
import re

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy

import init_titration
from titration import util as utl
from mfethuls import parse as pa

### Generate Titration from config

In [ ]:
# exp_names = ['LUB005', 'LUB100', 'LUB099', 'LUB055'] # Different metals (Zn, Eu, Cu) [all OTf] and endpoint
# exp_names = ['LUB005', 'LUB045', 'LUB067'] # Different metals (Zn, Eu, Cu) [all OTf] and order_complex=2
exp_names = ['LUB005', 'LUB021', 'LUB038', 'LUB192'] # Acids in Zn + TEAC
# exp_names = ['LUB005', 'LUB045', 'LUB055', 'LUB067', 'LUB157']
# exp_names = ['LUB005', 'LUB099', 'LUB045']
# exp_names = ['LUB045', 'LUB005', 'LUB157', 'LUB067', 'LUB055'] # Mebip decomplex hcl
# exp_names = ['LUB045', 'LUB005', 'LUB055'] # Mebip complex metals

In [ ]:
init_titration.init_titrations(exp_names)

In [ ]:
dict_titrants_exps = init_titration.dict_titrants_exps.copy()
dict_titrations_exps = deepcopy(init_titration.dict_titrations_exps)

In [ ]:
dict_titrants_exps

In [ ]:
for k, v in dict_titrations_exps.items():
    print(f'\n\n{k}:\n')
    for t in v[0]:
        print(t.df_params)

### Import data associated

In [ ]:
#TODO: Check parser get the first line of text file.

print(exp_names)
exps_reps = utl.get_exp_reps(exp_names)
data = pa.get_data(pa.path_constructor('uv', *exps_reps), 'uv')
data

### Distinguish from different titrations and merge titration params with data

In [ ]:
new_data = utl.merge_data_titration_params(data, dict_titrations_exps)
new_data

In [ ]:
new_data.titrant.unique()

### Generate peak tracking data

In [ ]:
# 350 nm for Eu
# 335 nm for Zn
# 345 nm for Cu
# 342 nm for Mg bidentate
# 345 nm for Cu bidentate
# 350 nm for Zn bidentate
# 350/495 nm for Fe2+ bidentate
# 555 nm for RhodB
# wl_0=313
dict_df = utl.tracking_df(new_data, [335], exps_reps, wl_0=313, complex_tit=True)
dict_df

In [ ]:
df = utl.combine_track_data(new_data, dict_df)
df

In [ ]:
if False:
    # Write this into the back - perhaps
    def remove_unrelated_wl_track(df, labels_mlct):
        for name, mlct in labels_MLCT.items():
            df.drop(df.loc[(df.name.str.contains(name)) & ~(df.wl_track == str(mlct))].index, inplace=True)
        df.reset_index(drop=True)
        return df
    
    # Name and associated mlct band wl for experiment
    # labels_MLCT = dict(zip(['LUB005', 'LUB045', 'LUB055', 'LUB067', 'LUB157'], [335, 350, 345, 345, 350]))
    # labels_MLCT = dict(zip(['LUB005', 'LUB193', 'LUB195'], [335, 350, 350])) # Mebip vs bidentate hcl
    # labels_MLCT = dict(zip(['LUB190', 'LUB194', 'LUB150'], [350, 350, 345])) # Mebip vs bidentate complex
    labels_MLCT = dict(zip(['LUB005', 'LUB045', 'LUB055', 'LUB067', 'LUB157', 'LUB193', 'LUB195'], 
                           [335, 350, 345, 345, 350, 350, 350])) # All Mebip and MBP decomplex
    df = remove_unrelated_wl_track(df, labels_MLCT)

### Visualise Data

##### Line Graphs

In [ ]:
# Legend used for graph - super manuel

# legend_labels = ['Eu(EH-Mebip)$_{2}$', 'Eu(EH-Mebip)$_{2}$ - Excees Eu$^{3+}$', 'Eu(EH-Mebip)$_{3}$',]
# legend_labels = ['Zn(EH-Mebip)$_{2}$', 'Eu(EH-Mebip)$_{2}$', 'Cu(EH-Mebip)$_{2}$']
# legend_labels = ['Zn(EH-Mebip)$_{2}$', 'Cu(EH-Mebip)', 'Eu(EH-Mebip)$_{2}$ - Excees Eu$^{3+}$']
legend_labels = ['HCl', 'TEAC', 'TFSA', 'TFA']
# legend_labels = ['Zn(EH-Mebip)$_{2}$', 'Eu(EH-Mebip)$_{2}$', 'Cu(EH-Mebip)$_{2}$', 'Cu(EH-Mebip)', 'Eu(EH-Mebip)$_{3}$']
# legend_labels = ['Eu(EH-Mebip)$_{2}$', 'Zn(EH-Mebip)$_{2}$', 'Eu(EH-Mebip)$_{3}$', 'Cu(EH-Mebip)', 
                 # 'Cu(EH-Mebip)$_{2}$']
# legend_labels = ['Eu(EH-Mebip)$_{2}$']
# legend_labels = ['Zn(EH-Mebip)$_{2}$']
# legend_labels = ['Cu(EH-Mebip)$_{2}$']
# legend_labels = ['Zn(OTf)$_{2}$']
# legend_labels = ['Eu(OTf)$_{3}$', 'Zn(OTf)$_{2}$', 'Cu(OTf)$_{2}$']
# legend_labels = ['Fe(EH-MBP)$_{3}$', 'Zn(EH-MBP)$_{3}$']
# legend_labels = ['Zn(OTf)$_{2}$', 'Fe(OTf)$_{3}$', 'Cu(OTf)$_{2}$']
# legend_labels = ['Fe(II)(OTf)$_{2}$', 'Fe(III)(OTf)$_{3}$']
# legend_labels = ['350 nm', '495 nm']
# legend_labels = ['HCl']
# legend_labels = ['100 min', '5 min']

# Designate grouper (to group titrations according to host or guest)
grouper = 'host_name'

In [ ]:
# Plot in seperate figures
# %matplotlib qt
# BG_WHITE = "#fafaf5"
with sns.plotting_context("paper", font_scale=1.75, 
                          rc={
                              "lines.linewidth": 2, 'lines.markersize': 8, 'patch.linewidth': 1.825
                          }):
    
    # Set colors for graph
    PalSet = sns.color_palette('deep', len(df.name.unique()))
    m_colors = PalSet.as_hex()
    m_colors = ['#ADE8F4', '#48CAE4', '#0096C7', '#0077B6', '#023E8A', '#96b1ad', '#439088'] # Mebip and MBP
    # m_colors = ['#96b1ad', '#439088', '#005852'] # MBP only
    
    # Plot data of interest
    to_plot=df[['g_h', 'host_name', 'guest_name', 'name', 'value', 'rank', 'wl_track']].drop_duplicates()
    to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]
    to_plot.sort_values(by='name', key=lambda column: column.map(lambda e: exp_names.index(e)), inplace=True) # Sort dataframe according to index provided

    for (exp, group) in to_plot.groupby([grouper], as_index=False, sort=False):
        print(exp)
        print(group.name.unique())
        print(group)
        
        fig, ax = plt.subplots() # Set a size once deteremined !
        # fig.patch.set_facecolor(BG_WHITE)
        # ax.set_facecolor(BG_WHITE)


        ax.set_xlabel('[Guest]:[Zn(EH-Mebip)$_{2}$]')
        ax.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontsize=20)
        ax.spines[['right', 'top']].set_visible(False)
        # ax.spines[['left', 'right', 'top']].set_visible(False)
        # ax.spines[['bottom']].set_linewidth(2)
        # ax.tick_params(length=4, axis='both')
        
        # Tick paramters
        # ax.xaxis.set_major_locator(MultipleLocator(0.5))
        # Metal regime
        # ax.xaxis.set_major_locator(MultipleLocator(0.1))
        # ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
        # ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
       
        # acid regime
        ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(1))
        ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
        ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
        # ax.tick_params(length=4, axis='both')
        # ax.tick_params(length=2, axis='both', which='minor')
        
        # ax.grid(axis='y', linestyle=':', lw=1, color='grey')
        # ax.tick_params(axis='y',length=0)
        ax.set_axisbelow(True)

        # Verticle lines indicating orders of complex for metal titrations
        # for el in [0.333, 0.5]:
        #     ax.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)
        # if group.host_name.unique()[0] == 'EH-Mebip':
        #     for el in [0.333, 0.5, 1]:
        #         ax.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)
        
        # lnplt = sns.lineplot(group, x="g_h", y="value", hue='name', ax=ax, palette='deep', legend=True)
        lnplt = sns.lineplot(group, x="g_h", y="value", hue='name', hue_order=exp_names, ax=ax, palette=m_colors, legend=True)
        
        # Legend - manual if legend required :(
        # new_title = 'Irradiation Time'
        bbox_to_anchor=(0.55, 0.45, 0.5, 0.5)
        # ax.legend(edgecolor='w', framealpha=0, loc='best')
        ax.legend(edgecolor='w', framealpha=0, bbox_to_anchor=bbox_to_anchor)
        # sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
        # lnplt.legend_.set_title(new_title)
        # Replace labels
        for text, label in zip(lnplt.legend_.texts, legend_labels):
            text.set_text(label)
        
        # Scatter plot for hollow spheres - matplotlib
        for (exp, group), col in zip(group.groupby(['name', 'guest_name', 'rank'], as_index=False, sort=False), m_colors):
            group_mean = group.groupby('g_h', as_index=False).agg({'value':'mean'})
            ax.scatter(group_mean.g_h, group_mean.value, edgecolors=col, facecolors="none", alpha=0.7, label=exp)

        fig.tight_layout()
        print(fig.get_size_inches())

##### Spectra

In [ ]:
%matplotlib qt
# Plots each titration individually
# Arange data and group plus plot

# TODO: Change pallete, remove legend
to_plot=df[['g_h', 'host_name', 'guest_name', 'name', 'Wavelength nm.', 'Abs.']].drop_duplicates()
to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]

with sns.plotting_context("paper", font_scale=1.75,
                          rc={
                              "lines.linewidth": 1.75, 'lines.markersize': 6.5, 'patch.linewidth': 1.35
                          }):

    for (exp, group) in to_plot.groupby(['name', grouper], as_index=False, sort=False):
        print(exp)
        print(group)
        if 'Eu' not in exp[0]:
            group = group.loc[group['Wavelength nm.'].between(250, 600), :]
            fig, ax = plt.subplots(1) # Set a size once deteremined !
        
            # Background colours
            # BG_WHITE = "#fafaf5"
            # fig.patch.set_facecolor(BG_WHITE)
            # ax.set_facecolor(BG_WHITE)
        
            # Border params
            ax.spines[['right', 'top']].set_visible(False)
            # ax.spines[['bottom']].set_linewidth(2)
            # ax.grid(axis='y', linestyle=':', lw=1, color='grey')
            # ax.set_axisbelow(True)
            
            # axis params
            ax.set_xlabel('Wavelength (nm)')
            ax.set_ylabel('Absorbance (a.u.)')
            ax.xaxis.set_minor_locator(AutoMinorLocator(2))
            ax.yaxis.set_major_locator(MultipleLocator(0.2))
            ax.yaxis.set_minor_locator(AutoMinorLocator(2))
            # ax.tick_params(which="both", bottom=True, top=True, left=True, right=True, direction="in")
            # ax.tick_params(which="both", labelbottom=True, labeltop=False, labelleft=True, labelright=False, direction="in")

            
            # Plot line graphs
            sns.lineplot(data=group, 
                         x="Wavelength nm.", y="Abs.", hue='g_h', palette='flare', ax=ax, legend=False)
            
            # Colourbar 
            norm = plt.Normalize(group.g_h.min(), group.g_h.max())
            sm = plt.cm.ScalarMappable(cmap='flare', norm=norm)
            cbar = fig.colorbar(sm, shrink=0.7, cax=ax.inset_axes((0.85, 0.125, 0.025, 0.75)))
            # cbar.ax.set_title(f'{exp[0]} eq.')
            # cbar.ax.set_title(f'HCl eq.')
            cbar.ax.set_title(f'Fe(III)(OTf)$_{2}$ eq.', pad=10)
            # cbar.ax.set_title(f'Dipea eq.', pad=10)
            
            # ax.set_ylim(ax.get_ylim()[0], 0.84)
            fig.tight_layout()

In [ ]:
# Plot spectra of complexed and decomplexed
with sns.plotting_context("paper"):
    fig, (ax, ax1) = plt.subplots(1, 2, figsize=(13.33/2, 7.5/2.5))
    ax.set_xlabel('Wavelength (nm)')
    
    # Remove right and top borders of plots
    ax.spines[['right', 'top', 'left']].set_visible(False)
    ax.spines[['bottom']].set_linewidth(2)
    ax.tick_params(length=4, axis='x')
    ax.grid(axis='y', linestyle=':', lw=1, color='grey')
    ax.tick_params(axis='y',length=0)
    ax.set_axisbelow(True)


to_plot=df[['g_h', 'guest_name', 'name', 'Wavelength nm.', 'Abs.']].drop_duplicates()
to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]
# to_plot.pivot(index='Wavelength nm.', columns=['name', 'g_h'], values='Abs.')
# Scatter plot for hollow spheres - matplotlib
to_plot = to_plot[to_plot.name == 'LUB045']
for (exp, group) in to_plot.groupby(['name', 'guest_name'], as_index=False):
    fig, ax = plt.subplots(1)
    sns.lineplot(data=group, x="Wavelength nm.", y="Abs.", hue='g_h')
    # group_mean = group.groupby('g_h', as_index=False).agg({'value':'mean'})
    # print(f'{exp} : {col}')
    # ax.scatter(group_mean.g_h, group_mean.value, edgecolors=col, facecolors="none", alpha=0.7, label=exp)
# sns.lineplot(to_plot, x="Wavelength nm.", y="Abs.", hue='g_h', style='name')